In [23]:
import glob
from dotenv import load_dotenv
import os
from chroma_utils import PrepareDocumentsFolder,VectorStore
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [24]:
load_dotenv()
DB_NAME = os.getenv("VECTOR_DB_NAME")
OPENAI_MODEL = os.getenv("OPENAI_MODEL")

In [25]:
folders = glob.glob("knowledge-base/*")

In [26]:
prepared_documents = PrepareDocumentsFolder(folders)
chunks = prepared_documents.create_chunks()

2025-05-05 18:31:34,691 - langchain_text_splitters.base - WARNING - Created a chunk of size 1088, which is longer than the specified 1000
2025-05-05 18:31:34,692 - utils.tools - INFO - Document types found: contracts, employees, company, products


In [27]:
vectorstore = VectorStore(chunks,DB_NAME)

2025-05-05 18:31:34,711 - utils.tools - INFO - Deleted existing collection at vector-db
2025-05-05 18:31:36,101 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 18:31:37,120 - utils.tools - INFO - Vectorstore created with 123 documents
2025-05-05 18:31:37,122 - utils.tools - INFO - Vectorstore 'vector-db' has 1,536 dimensions


In [30]:

llm = ChatOpenAI(temperature=0.7, model_name=OPENAI_MODEL)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [31]:
manual_query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":manual_query})
print(result["answer"])

2025-05-05 18:32:25,175 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 18:32:27,527 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Insurellm is an innovative insurance tech startup founded in 2015 by Avery Lancaster, aimed at disrupting the insurance industry with its technology-driven products. The company offers four main software products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. With 200 employees and over 300 clients worldwide, Insurellm is committed to transforming the insurance landscape through innovation and reliability.


In [32]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [33]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

2025-05-05 18:32:34,359 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7875/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-05-05 18:32:34,368 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7875/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


2025-05-05 18:32:34,849 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-05-05 18:32:44,910 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 18:32:45,249 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 18:32:46,086 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 18:33:23,049 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-05 18:33:23,836 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-05 18:33:28,761 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
